In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.1,
                                 zoom_range=0.1,
                                 horizontal_flip=True)

val_datagen=ImageDataGenerator(rescale=1./255)

train_transform=train_datagen.flow_from_directory(r"C:\Users\baliv\Desktop\garbage collection\Dataset\train",

                                                 target_size=(128, 128),
                                                 batch_size=64,
                                                 class_mode='categorical')

test_transform=val_datagen.flow_from_directory(r"C:\Users\baliv\Desktop\garbage collection\Dataset\test",
                                        
                                                 target_size=(128, 128),
                                                 batch_size=64,
                                                 class_mode='categorical')

Found 4105 images belonging to 6 classes.
Found 120 images belonging to 6 classes.


In [5]:
val_datagen = ImageDataGenerator(rescale=1./255)  # You can apply other augmentation settings if needed.

val_transform = val_datagen.flow_from_directory(r"C:\Users\baliv\Desktop\garbage collection\Dataset\val",
                                               target_size=(128, 128),
                                               batch_size=64,
                                               class_mode='categorical')

Found 481 images belonging to 6 classes.


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Flatten())

# Add the output layer with 6 units (for 6 classes) and use softmax activation for multi-class classification
model.add(Dense(6, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

res = model.fit(
    train_transform,
    steps_per_epoch=len(train_transform),
    epochs=10,
    validation_data=val_transform,
    validation_steps=len(val_transform)
)

test_steps = len(test_transform)
test_loss, test_accuracy = model.evaluate(test_transform, steps=test_steps)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 63, 63, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 31, 31, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 29, 29, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 32)        0

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import ResNet50

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_transform = train_datagen.flow_from_directory(
    r"C:\Users\baliv\Desktop\garbage collection\Dataset\train",
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical'
)

val_transform = val_datagen.flow_from_directory(
    r"C:\Users\baliv\Desktop\garbage collection\Dataset\val",
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical'
)

# Model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.summary()

# Learning Rate Schedule
def lr_schedule(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * 0.1

lr_scheduler = LearningRateScheduler(lr_schedule)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])

# Train the model
res = model.fit(
    train_transform,
    steps_per_epoch=len(train_transform),
    epochs=25,
    validation_data=val_transform,
    validation_steps=len(val_transform),
    callbacks=[lr_scheduler]
)
test_steps = len(test_transform)
test_loss, test_accuracy = model.evaluate(test_transform, steps=test_steps)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Found 4105 images belonging to 6 classes.
Found 481 images belonging to 6 classes.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 batch_normalization (Batch  (None, 63, 63, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_5 (Conv2D)           (None, 63, 63, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 31, 31, 64)        0         
 g2D)                                

In [8]:
model.save("garbage_classification_model.h5")

C:\Users\baliv\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    rotation_range=20,  # Additional augmentation
    width_shift_range=0.1,  # Additional augmentation
    height_shift_range=0.1  # Additional augmentation
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_transform = train_datagen.flow_from_directory(
    r"C:\Users\baliv\Desktop\garbage collection\Dataset\train",
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical'
)

val_transform = val_datagen.flow_from_directory(
    r"C:\Users\baliv\Desktop\garbage collection\Dataset\val",
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical'
)

# Model using ResNet50
model = Sequential()
resnet_model = ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
model.add(resnet_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.summary()

# Learning Rate Schedule
def lr_schedule(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * 0.1

lr_scheduler = LearningRateScheduler(lr_schedule)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])

# Train the model
res = model.fit(
    train_transform,
    steps_per_epoch=len(train_transform),
    epochs=25,
    validation_data=val_transform,
    validation_steps=len(val_transform),
    callbacks=[lr_scheduler]
)

# Save the model
model.save('garbage_classification_model.h5')

Found 4105 images belonging to 6 classes.
Found 481 images belonging to 6 classes.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten_2 (Flatten)         (None, 32768)             0         
                                                                 
 dense_3 (Dense)             (None, 256)               8388864   
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 31978118 (121.99 MB)
Trainable params: 31924998 (121.78 MB)
Non-trainable params: 53120 (

In [17]:
model.save('garbage_classification_model.h5')

In [18]:
test_steps = len(test_transform)
test_loss, test_accuracy = model.evaluate(test_transform, steps=test_steps)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

2/2 [==============================] - 2s 862ms/step - loss: 0.4238 - acc: 0.8500
Test Loss: 0.4238184094429016
Test Accuracy: 0.8500000238418579
